In [2]:
from collections import Counter
import numpy as np
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,ConfusionMatrixDisplay
from tensorflow.keras.layers import Input, Dense,Concatenate, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D,Dropout, Add,Embedding
from keras.models import Model, load_model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
MAX_URL_LEN_CHAR = 200
MAX_URL_LEN_WORD = 200
MAX_WORD_LEN = 20
k = 32
num_filters = 256

In [4]:
conn = sqlite3.connect('casino.db')

df = pd.read_sql('select * from web_pages',conn)
df.drop(columns=["id","url","html","screenshot"],inplace=True)

In [5]:
urls = df.real_url.values

In [6]:
urls[0]

'https://bc.game/?stag=6954_666ec6ad9782f44562dc5ce0&i=4cxse6dr&utm_source=4cxse6dr'

# Character level CNN

## Character level tokenizer

In [7]:
# Character tokenization
char_tokenizer = Tokenizer(char_level=True, oov_token='<OOV>')
char_tokenizer.fit_on_texts(urls)

char_sequences = char_tokenizer.texts_to_sequences(urls)
char_padded = pad_sequences(char_sequences, maxlen=200, padding='post')

In [8]:
input_shape = char_padded.shape[1]
print(f'input shape: {input_shape}')

input shape: 200


### Character embedding

In [9]:
char_input = Input(shape=(MAX_URL_LEN_CHAR,))

# Embedding layer
char_embedding = Embedding(input_dim=len(char_tokenizer.word_index) + 1, output_dim=k, input_length=MAX_URL_LEN_CHAR)(char_input)
char_embedding = tf.expand_dims(char_embedding, -1)  # Add channel dimension

### CNN Block

In [10]:
# Convolution and pooling for character-level
# h = 3
conv_3_char = Conv2D(num_filters, (3, k), activation='relu')(char_embedding)
conv_3_char = MaxPooling2D((2, 1), strides=(2, 1))(conv_3_char)
# h = 4
conv_4_char = Conv2D(num_filters, (4, k), activation='relu')(char_embedding)
conv_4_char = MaxPooling2D((2, 1), strides=(2, 1))(conv_4_char)
# h = 5
conv_5_char = Conv2D(num_filters, (5, k), activation='relu')(char_embedding)
conv_5_char = MaxPooling2D((2, 1), strides=(2, 1))(conv_5_char)
# h = 6
conv_6_char = Conv2D(num_filters, (6, k), activation='relu')(char_embedding)
conv_6_char = MaxPooling2D((2, 1), strides=(2, 1))(conv_6_char)

### Fully connected layer

In [11]:
#concatenate all convolutional layer outputs
concatenated = Concatenate(axis=1)([conv_3_char,conv_4_char,conv_5_char,conv_6_char])
flattened = Flatten()(concatenated)

# feed concatenated conv layers to fully conected layer
dense_char = Dense(1024,activation='relu')(flattened)
dropout = Dropout(0.5)(dense_char)

# Word-level CNN

### Word level tokenizer

In [12]:
# Word tokenization
word_tokenizer = Tokenizer(oov_token='<OOV>')
word_tokenizer.fit_on_texts(urls)

word_sequences = word_tokenizer.texts_to_sequences(urls)

#### Calculate number of words which appear more than once

In [13]:
all_sequences = [sequence for sequence in word_tokenizer.sequences_to_texts(word_sequences)]
all_words_concat = ' '.join([' '.join(sequence.split()) for sequence in all_sequences])

freq_dict_words = Counter(all_words_concat.split())

low_freq_words_count = 0
for key in freq_dict_words:
    if freq_dict_words[key] == 1:
        low_freq_words_count += 1

high_freq_word_count = int(len(freq_dict_words) * (1-(low_freq_words_count/len(freq_dict_words))))
print(high_freq_word_count)

318


#### Url into sequence of word ids

In [14]:
# reinstaniate tokenizer
word_tokenizer = Tokenizer(oov_token='<OOV>',num_words=high_freq_word_count)
word_tokenizer.fit_on_texts(urls)
print(len(word_tokenizer.word_counts))

word_sequences = word_tokenizer.texts_to_sequences(urls)
word_padded = pad_sequences(word_sequences, maxlen=MAX_URL_LEN_WORD, padding='post')

1861


In [15]:
word_padded

array([[2, 1, 1, ..., 0, 0, 0],
       [2, 4, 1, ..., 0, 0, 0],
       [2, 4, 1, ..., 0, 0, 0],
       ...,
       [2, 1, 3, ..., 0, 0, 0],
       [2, 4, 1, ..., 0, 0, 0],
       [2, 4, 1, ..., 0, 0, 0]])

(506, 200)

#### url into sequence of words in char ids 

In [19]:
word_tokenizer_for_chars = Tokenizer(oov_token='<OOV>')
word_tokenizer_for_chars.fit_on_texts(urls)

sentences_to_sequences = word_tokenizer_for_chars.texts_to_sequences(urls)
sentences_padded = pad_sequences(sentences_to_sequences,maxlen=MAX_URL_LEN_WORD,padding='post')
words_splitted = word_tokenizer_for_chars.sequences_to_texts(sentences_padded)

words_splitted = [sentence.split() for sentence in words_splitted]
words_splitted = [[word.split() for word in sentence] for sentence in words_splitted]

word_char_sequences = []

for sentence in words_splitted:
    sentence_tokenized = []
    for i in range(len(sentence)):
        word = char_tokenizer.texts_to_sequences(sentence[i])
        word_char_padded = pad_sequences(word,maxlen=MAX_WORD_LEN,padding='post')
        sentence_tokenized.append(word_char_padded)
    word_char_sequences.append(sentence_tokenized)

word_char_sequences = np.array(word_char_sequences)
word_char_sequences = word_char_sequences.reshape(word_char_sequences.shape[0],word_char_sequences.shape[1],word_char_sequences.shape[-1])



(506, 200, 20)

##### word embedding

In [24]:
word_input = Input(shape=(MAX_URL_LEN_WORD,))

# Embedding layer
word_embedding = Embedding(input_dim=len(word_tokenizer.word_index) + 1, output_dim=k, input_length=MAX_URL_LEN_WORD)(word_input)
# word_embedding = tf.expand_dims(word_embedding, -1)  # Add channel dimension

In [25]:
word_embedding.shape

TensorShape([None, 200, 32])

##### char embedding 2

In [26]:
word_char_input = Input(shape=(MAX_URL_LEN_CHAR,MAX_WORD_LEN))

# Embedding layer
word_char_embedding = Embedding(input_dim=len(char_tokenizer.word_index) + 1, output_dim=k, input_length=MAX_URL_LEN_CHAR)(word_char_input)
# word_char_embedding = tf.expand_dims(word_char_embedding, -1)  # Add channel dimension

In [27]:
word_char_embedding.shape

TensorShape([None, 200, 20, 32])

##### Sum over characters

In [28]:
pooled_layer = tf.keras.layers.Lambda(lambda x: tf.math.reduce_sum(x, axis=2))(word_char_embedding)


In [30]:
pooled_layer.shape

TensorShape([None, 200, 32])

#### Element-wise Addition

In [33]:
addition_layer = Add()([pooled_layer, word_embedding])
addition_layer = tf.expand_dims(addition_layer,-1)

In [34]:
addition_layer.shape

TensorShape([None, 200, 32, 1])

## CNN Block

In [35]:
# Convolution and pooling for word-level
# h = 3
conv_3_word = Conv2D(num_filters, (3, k), activation='relu')(addition_layer)
conv_3_word = MaxPooling2D((2, 1), strides=(2, 1))(conv_3_word)
# h = 4
conv_4_word = Conv2D(num_filters, (4, k), activation='relu')(addition_layer)
conv_4_word = MaxPooling2D((2, 1), strides=(2, 1))(conv_4_word)
# h = 5
conv_5_word = Conv2D(num_filters, (5, k), activation='relu')(addition_layer)
conv_5_word = MaxPooling2D((2, 1), strides=(2, 1))(conv_5_word)
# h = 6
conv_6_word = Conv2D(num_filters, (6, k), activation='relu')(addition_layer)
conv_6_word = MaxPooling2D((2, 1), strides=(2, 1))(conv_6_word)

## Word level fully connected layer

In [36]:
#concatenate all convolutional layer outputs
concatenated_word = Concatenate(axis=1)([conv_3_word,conv_4_word,conv_5_word,conv_6_word])
flattened_word = Flatten()(concatenated_word)

# feed concatenated conv layers to fully conected layer
dense_word = Dense(1024,activation='relu')(flattened_word)
dropout_word = Dropout(0.5)(dense_word)

In [37]:
dropout_word.shape

TensorShape([None, 1024])

## concatenate outputs of char-level and word-level blocks

In [42]:
concatenate_all = Concatenate()([dropout,dropout_word])
concatenate_all.shape

TensorShape([None, 2048])

# last fully connected layers

In [45]:
fc_1 = Dense(512,activation='relu')(concatenate_all)
dropout_fc_1 = Dropout(0.5)(fc_1)

fc_2 = Dense(256,activation='relu')(fc_1)
dropout_fc_2 = Dropout(0.5)(fc_2)

fc_3 = Dense(128,activation='relu')(fc_2)
dropout_fc_3 = Dropout(0.5)(fc_3)
dropout_fc_3.shape

TensorShape([None, 128])

#### output layer with softmax

In [ ]:
def charTokenize(string):
    char_list = []
    for char in string:
        char_list.append(char)
    return char_list
concat_urls = ''.join([url.lower() for url in urls])

chars = sorted(list(set(concat_urls)))
char_vocab_size = len(chars)
print(char_vocab_size)

char_to_int = {char:i for i,char in enumerate(chars)}
int_to_char = {i:char for i,char in enumerate(chars)}
encode = lambda url: [char_to_int[ch] for ch in charTokenize(url)] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([int_to_char[i] for i in l]) # decoder: take a list of integers, output a string

decode(encode(urls[0]))
def encode_url(url):
    url = [char_to_int[ch] for ch in charTokenize(url)]
    while len(url<200):
        url.append()
    return url